# Bayesian GP Model for IEDDA Reaction Rates

In [ ]:
!pip install rdkit pymc arviz --quiet

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pymc as pm
import arviz as az
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdPartialCharges
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from google.colab import files


## Upload CSV Data

In [ ]:
uploaded = files.upload()
csv_path = next(iter(uploaded))
df = pd.read_csv(csv_path)

df = df.rename(columns={
    'isomeric SMILES(NB)': 'smiles_NB',
    'isomeric SMILES(Tz)': 'smiles_Tz',
    'dielectric constant': 'dielectric',
    'ET(30), kcal/mol': 'ET30',
    'temperature(K)': 'temp_K',
    'Rate Constant (k₂) M-1s-1': 'rate_constant',
    'error': 'error'
})
df.head()


## Calculate Molecular Descriptors

In [ ]:
def compute_desc(smiles, error_log):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        error_log.append(smiles)
        return (np.nan,)*6
    try:
        rdPartialCharges.ComputeGasteigerCharges(mol)
        return (
            Descriptors.MolWt(mol),
            Descriptors.MolLogP(mol),
            Descriptors.TPSA(mol),
            Descriptors.MaxPartialCharge(mol),
            Descriptors.LabuteASA(mol),
            Descriptors.BalabanJ(mol),
        )
    except Exception as e:
        error_log.append(f"{smiles} ({e})")
        return (np.nan,)*6

nb_error_log, tz_error_log = [], []

nb_feats = df['smiles_NB'].apply(lambda x: compute_desc(x, nb_error_log)).tolist()
tz_feats = df['smiles_Tz'].apply(lambda x: compute_desc(x, tz_error_log)).tolist()

df[['NB_MolWt', 'NB_LogP', 'NB_TPSA', 'NB_MaxPartialCharge', 'NB_LabuteASA', 'NB_BalabanJ']] = pd.DataFrame(nb_feats, index=df.index)
df[['Tz_MolWt', 'Tz_LogP', 'Tz_TPSA', 'Tz_MaxPartialCharge', 'Tz_LabuteASA', 'Tz_BalabanJ']] = pd.DataFrame(tz_feats, index=df.index)


## PCA Feature Reduction

In [ ]:
tz_cols = ['Tz_MolWt', 'Tz_LogP', 'Tz_TPSA', 'Tz_MaxPartialCharge', 'Tz_LabuteASA', 'Tz_BalabanJ']
nb_cols = ['NB_MolWt', 'NB_LogP', 'NB_TPSA', 'NB_MaxPartialCharge', 'NB_LabuteASA', 'NB_BalabanJ']
other_cols = ['dielectric', 'ET30', 'temp_K']

pca_input_cols = nb_cols + tz_cols + other_cols + ['rate_constant', 'error']
df = df.dropna(subset=pca_input_cols)

def scale_pca(df, cols):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df[cols])
    pca = PCA(n_components=2)
    return pca.fit_transform(X_scaled), scaler, pca

tz_pca_result, scaler_tz, pca_tz = scale_pca(df, tz_cols)
nb_pca_result, scaler_nb, pca_nb = scale_pca(df, nb_cols)
other_pca_result, scaler_other, pca_other = scale_pca(df, other_cols)

df['Tz_PC1'], df['Tz_PC2'] = tz_pca_result[:, 0], tz_pca_result[:, 1]
df['NB_PC1'], df['NB_PC2'] = nb_pca_result[:, 0], nb_pca_result[:, 1]
df['Other_PC1'], df['Other_PC2'] = other_pca_result[:, 0], other_pca_result[:, 1]

feature_cols = ['Tz_PC1', 'Tz_PC2', 'NB_PC1', 'NB_PC2', 'Other_PC1', 'Other_PC2']
scaler_final = StandardScaler()
X_scaled = scaler_final.fit_transform(df[feature_cols])
y = df['rate_constant'].values
sigma_obs = df['error'].clip(lower=0.02).values


## Bayesian Gaussian Process Model with PyMC

In [ ]:
with pm.Model() as gp_model:
    ℓ = pm.Gamma("ℓ", alpha=5, beta=2)
    η = pm.HalfNormal("η", sigma=1)
    cov = η**2 * pm.gp.cov.ExpQuad(input_dim=X_scaled.shape[1], ls=ℓ)

    gp = pm.gp.Latent(cov_func=cov)
    f = gp.prior("f", X=X_scaled)

    sigma_known = sigma_obs
    sigma_total = pm.Deterministic("sigma_total", pm.math.sqrt(sigma_obs**2 + sigma_known**2))
    y_ = pm.Lognormal("y", mu=f, sigma=sigma_total, observed=y)


## Posterior Sampling with NUTS

In [ ]:
with gp_model:
    trace = pm.sample(
        draws=2000,
        tune=1000,
        chains=2,
        target_accept=0.95,
        random_seed=42
    )


## Trace and Posterior Plots

In [ ]:
az.plot_trace(trace, var_names=["ℓ", "η"])
plt.tight_layout()
plt.show()

az.plot_posterior(trace, var_names=["ℓ", "η"])
plt.tight_layout()
plt.show()


## Posterior Predictive Check

In [ ]:
with gp_model:
    idata_ppc = pm.sample_posterior_predictive(trace, var_names=["y"], random_seed=42, return_inferencedata=True)

y_ppc_log = idata_ppc.posterior_predictive["y"].stack(sample=("chain", "draw")).values
y_ppc = np.exp(y_ppc_log)
mu_ppc = y_ppc.mean(axis=-1)
sd_ppc = y_ppc.std(axis=-1)

plt.figure(figsize=(10, 5))
plt.errorbar(np.arange(len(mu_ppc)), mu_ppc, yerr=sd_ppc, fmt='o', label="Posterior Predictive Mean ± SD")
plt.scatter(np.arange(len(y)), y, color='r', alpha=0.7, label="Observed y")
plt.xlabel("Observation Index")
plt.ylabel("y value")
plt.legend()
plt.title("Observed vs Posterior Predictive Mean ± SD")
plt.tight_layout()
plt.show()
